In [1]:
from pathlib import Path

import pandas as pd

In [2]:
dates_to_visualize = [
                                                                                            "2018-09", "2018-10", "2018-11", "2018-12",
    "2019-01", "2019-02", "2019-03", "2019-04", "2019-05", "2019-06", "2019-07", "2019-08", "2019-09", "2019-10", "2019-11", "2019-12", 
    "2020-01", "2020-02", "2020-03", "2020-04", "2020-05", "2020-06", "2020-07", "2020-08", "2020-09", "2020-10", "2020-11", "2020-12",
    "2021-01", "2021-02", "2021-03", "2021-04", "2021-05", "2021-06", "2021-07", "2021-08", "2021-09", "2021-10", "2021-11", "2021-12",
    "2022-01", "2022-02", "2022-03", "2022-04", "2022-05", "2022-06", "2022-07", "2022-08", "2022-09", "2022-10", "2022-11", "2022-12",
    "2023-01", "2023-02",
    # "2024-02",
]

In [3]:
available_data_files = list(Path("../data/httparchive_metrics/nel_monitored_resource_types").glob("*.parquet"))

used_data_files = list(filter(lambda file: file.stem in dates_to_visualize, available_data_files))
used_data_files

[WindowsPath('../data/httparchive_metrics/nel_monitored_resource_types/2018-09.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_monitored_resource_types/2018-10.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_monitored_resource_types/2018-11.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_monitored_resource_types/2018-12.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_monitored_resource_types/2019-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_monitored_resource_types/2019-03.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_monitored_resource_types/2019-04.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_monitored_resource_types/2019-05.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_monitored_resource_types/2019-06.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_monitored_resource_types/2019-07.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_monitored_resource_types/2019-08.parquet'),
 WindowsPa

In [4]:
from results.result_utils import date_to_text_format, concat_data_from_files

result = concat_data_from_files(used_data_files)

result['date_formatted'] = result['date'].map(date_to_text_format)

result

,date,url_domain,type,count,date_formatted
0,2018-09,m2.cn.bing.com,html,1,Sep 2018
1,2018-09,www4.bing.com,html,1,Sep 2018
0,2018-10,classifieds.seloc.org,image,52,Oct 2018
1,2018-10,classifieds.seloc.org,script,12,Oct 2018
2,2018-10,classifieds.seloc.org,html,1,Oct 2018
...,...,...,...,...,...
8153484,2023-02,zzzzap.nl,css,20,Feb 2023
8153485,2023-02,zzzzap.nl,font,2,Feb 2023
8153486,2023-02,zzzzzz.me,html,4,Feb 2023
8153487,2023-02,zzzzzz.me,script,1,Feb 2023


### Monitored resources by type

In [5]:
monthly_data = pd.DataFrame()
for month in dates_to_visualize:
    current_month_data = result[result['date'] == month].copy()
    
    current_month_data = current_month_data.groupby(['date', 'type'], as_index=False).agg({'count': 'sum'})
    current_month_data.sort_values(by='count', ascending=False, inplace=True)
    current_month_data.reset_index(inplace=True, drop=True)
    
    current_month_data['date'] = current_month_data['date'].map(date_to_text_format)
    
    monthly_data = pd.concat([monthly_data, current_month_data])
    
# TODO analyze on domains that have a % of NEL monitored resources < 100% (lower ranges 10%-20%)
    
monthly_data

,date,type,count
0,Sep 2018,html,2
0,Oct 2018,image,57
1,Oct 2018,script,15
2,Oct 2018,html,13
3,Oct 2018,css,4
...,...,...,...
5,Feb 2023,other,2178610
6,Feb 2023,text,393887
7,Feb 2023,video,327562
8,Feb 2023,xml,45898


### Most monitored resource type for the whole time period (HTTPArchive data) 

In [6]:
result.groupby('type')['count'].sum().sort_values(ascending=False)

type
image     1073928564
script     515131842
css        245776190
html        89222382
font        55915325
other       32199103
text         5495479
video        3748888
xml           953086
audio         812891
Name: count, dtype: UInt32